# Extracting Data from a PDF with Tabula
The purpose of this notebook is to demonstrate how to import data from a pdf and create a csv (or other format) file with the contents. We'll use **Tabula** for this job. Python has a package that acts as a wrapper to the Java Tabula program and there is a requirement to have a Java runtime available. For assistance with your platform, follow the instructions at:
- https://pypi.org/project/tabula-py/
- https://nbviewer.jupyter.org/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb

For demonstration purposes we'll use the pdf's made publically available by the Nova Scotia government which contains the results of water testing for the schools in Nova Scotia.

Source of PDF's:
https://www.ednet.ns.ca/water-test-results


The government site provides eight separate files, one for each of the regions. The first four pages are a similar bit of text, then the data starts. We can read directly from the website, however, we'll first download and save the files locally in case they move, become unavailable or change in format in the future.

In [1]:
# create a list of regions (named based on the files on the gov site)
regions = ['avrce',
           'cbvrce',
           'ccrce',
           'csap_en', # assume there is a French equivalent
           'hrce',
           'srce',
           'ssrce',
           'tcrce']

In [ ]:
# NOTE: after you've run this cell once, you don't need to run it everytime as you 
# already have the files locally saved (unless you want updates)
import requests

# using the requests (https://requests.readthedocs.io/) library, download the pdf's and save a copy locally
for region in regions:
    url = f"https://www.ednet.ns.ca/sites/default/files/docs/watertestresults_{region}.pdf"
    with requests.get(url, allow_redirects=True) as r:
        with open(f"watertestresults_{region}.pdf", 'wb') as local_copy:
            local_copy.write(r.content)


## Extract raw text
It is possible to obtain the raw text in the pdf. I've tried a number of method, but find [**pdftotext**](https://pypi.org/project/pdftotext/) to be the most reliable. Others like [**PyPDF2**](https://pypi.org/project/PyPDF2/) and subsequent forks omit some of the text. I've not been able to determine why, but notice consistently that the same blocks of text are missing from the output. In this sample, it was the first and last bullet points on the forth page. **PDFToText** seems to get all the text everytime, however, for tabular data, I find **Tabula** does a wonderful job.

In [2]:
# optional:
# for interest sake, pull the text from the first four pages and print it here
import pdftotext
with open("watertestresults_srce.pdf", "rb") as f:
    pdf = pdftotext.PDF(f,raw=False)
for page in range(4):
    print(pdf[page])

Water Test Results
Strait Regional Centre for Education
                                     Prepared by the Department of Education and Early Childhood Development, September 2020

Introduction
In December of 2019, the Province of Nova Scotia committed to testing the water in all public schools
across Nova Scotia after Health Canada revised its guidelines related to lead concentration levels in water.
Health Canada also revised its guidelines related to copper concentration levels in water, and copper tests
were also conducted. To ensure that parents were confident in the availability of safe drinking water, the
government also put bottled water in every public school in Nova Scotia. This water will remain in place
until appropriate mediation steps are taken.
In the meantime, Regional Centres of Education (RCEs) and Conseil scolaire acadien provincial (CSAP)
have taken steps to remove access to any other water source until test results were received, and
appropriate remediation steps 

## No Guarantee
The purpose of this notebook is to show a practical example using Tabula to extract data. We'll also plan to conflate the data from the eight PDF files into a single csv file for download. No guarantee is made on the completeness of this data as stored here in PDF, or as extracted and saved back in raw form. As discussed above, extracting data from PDF's can be troublesome and it is up to the reader to quality check the output for integrity, completeness and accuracy.

After going through the process below it was noted that 20 records are not properly imported from the avrce file. The records' location spans multiple lines, and in some cases results in a blank, in others a description that only has the second line in the Location text. Data is like this sometimes and may require some decisions afterward. Do we clean up manually, remove all blanks or come up with another solution?

## **NOTE:** this next cell may take a couple of minutes to complete
But it does report progress below it as it goes.

In [3]:
import pandas as pd
import tabula

# create an empty dataframe (df) which we'll append each region's data
df = pd.DataFrame()

for region in regions:
    pdf_path = f"watertestresults_{region}.pdf"
    tabula_df = tabula.read_pdf(pdf_path, pages='all', stream=True)
    
    temp_df = pd.DataFrame()
    for page in tabula_df:
        temp_df = temp_df.append(page.set_index("Sample ID"))
    # since the pdf file doesn't include the region, we'll add that column and fill it
    # NB: there is probably a more memory efficient way to accomplish this
    temp_df['Region'] = region
    df = df.append(temp_df)
    print(f"Imported {len(temp_df.index)} records for {region}. Total in df: {len(df.index)}")
    

Imported 2182 records for avrce. Total in df: 2182
Imported 1134 records for cbvrce. Total in df: 3316
Imported 1939 records for ccrce. Total in df: 5255
Imported 470 records for csap_en. Total in df: 5725
Imported 6306 records for hrce. Total in df: 12031
Imported 312 records for srce. Total in df: 12343
Imported 1252 records for ssrce. Total in df: 13595
Imported 1163 records for tcrce. Total in df: 14758


In [4]:
# look at the data types
df.dtypes

School Name         object
Location            object
Lead (Pb) mg/L      object
Lead Result         object
Copper (Cu) mg/L    object
Copper Result       object
Region              object
dtype: object

In [5]:
# and recognize we need to convert the mg/L columns to numeric
df['Lead (Pb) mg/L'] = pd.to_numeric(df['Lead (Pb) mg/L'], errors='coerce')
df['Copper (Cu) mg/L'] = pd.to_numeric(df['Copper (Cu) mg/L'], errors='coerce')
df.dtypes

School Name          object
Location             object
Lead (Pb) mg/L      float64
Lead Result          object
Copper (Cu) mg/L    float64
Copper Result        object
Region               object
dtype: object

## What to do now?
Now the data is in our dataframe what can we do with it? We can use Panda's to analyse it, export it to Excel, csv or other formats, plot charts and run any other sort of analyse we like. This is where the learning curve comes into play - Pandas is a great tool, but does require some learning. There are a number of online courses available; some high quality ones on [Pluralsight](https://www.pluralsight.com/).

What follows is some examples what sort of things can be done.

In [6]:
# export to excel
# see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
# for options
df.to_excel("water_test_results.xlsx")

In [7]:
# save an excel file with only records that have a sample id
df[df.index.notnull()].to_excel("water_test_results_noblanks.xlsx")

In [8]:
# similar for csv
df.to_csv("water_test_results.csv")

In [9]:
# find a particular Record
df[df.index=="MXH510"]

,School Name,Location,Lead (Pb) mg/L,Lead Result,Copper (Cu) mg/L,Copper Result,Region
Sample ID,,,,,,,
MXH510,Prince Andrew High School,NaN,0.0072,Exceeds Limit,2.6,Exceeds Limit,hrce


In [10]:
# see the records that have the additional second line in the Locations
# note that NaN is pandas way to say Not A Number 
df[df["School Name"].isnull()]

,School Name,Location,Lead (Pb) mg/L,Lead Result,Copper (Cu) mg/L,Copper Result,Region
Sample ID,,,,,,,
NaN,NaN,M/E Guidence Office W/R - Main Entrance Guiden...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Janitor Room across from Room 102 - Janitor Ro...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Janitor Room-New Gym Girls L/R - Janitor Room-...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Boys W/R across from Custodial Office - Boys W...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Boys W/R across from Custodial Office - Boys W...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Boys W/R across from Custodial Office - Boys W...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Girls W/R across from Custodial Office - Girls...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Girls W/R across from Custodial Office - Girls...,NaN,NaN,NaN,NaN,avrce
NaN,NaN,Girls W/R across from Custodial Office - Girls...,NaN,NaN,NaN,NaN,avrce


In [11]:
# list all the regions in our df
for reg in sorted(df["Region"].unique()):
    print(reg)

avrce
cbvrce
ccrce
csap_en
hrce
srce
ssrce
tcrce


In [12]:
# list unique school names
for school in df["School Name"].dropna().unique():
    print(school)

# messy way to remove the ones with null values
# schools = [school for school in df["School Name"][-df["School Name"].isnull()].unique()]
# print(sorted(schools))

Aldershot Elementary School
Annapolis East Elementary School
Annapolis West Education Centre
Avon View High School
Berwick & District School
Bridgetown Regional Community School
Brooklyn District Elementary School
Cambridge & District Elementary School
Central Kings Rural High School
Champlain Elementary School
Clark Rutherford Memorial School
Coldbrook & District School
Dr. Arthur Hines Elementary School
Dwight Ross Elementary School
Evangeline Middle School
Falmouth District School
Gaspereau Valley Elementary School
Glooscap Elementary School
Hantsport School
Horton High School
Kings County Academy
Kingston & District School
L.E. Shaw Elementary School
Lawrencetown Consolidated School
Lawrencetown Education Centre
Middleton Bus Garage
Middleton Regional High School
New Minas Elementary School
Northeast Kings Education Centre
Operations Centre
Pine Ridge Middle School
Port Williams Elementary School
Somerset & District Elementary School
St. Mary's Elementary School
Three Mile Plains D

In [13]:
# quick description for the entire data set
df.describe()

,Lead (Pb) mg/L,Copper (Cu) mg/L
count,13603.000000,13603.000000
mean,0.022222,0.416194
std,0.235173,1.053212
min,0.000000,0.000000
25%,0.000700,0.081000
50%,0.002200,0.180000
75%,0.008200,0.410000
max,20.000000,53.000000


In [14]:
# percentile list 
perc =[.20, .40, .60, .80] 
  
# list of dtypes to include 
include =['float', 'int']

df.dropna().describe(percentiles=perc, include=include)

,Lead (Pb) mg/L,Copper (Cu) mg/L
count,13530.000000,13530.000000
mean,0.021983,0.414757
std,0.235502,1.053373
min,0.000000,0.000000
20%,0.000400,0.064000
40%,0.001300,0.133220
50%,0.002200,0.180000
60%,0.003700,0.240000
80%,0.011000,0.506700
max,20.000000,53.000000


In [15]:
# making boolean series for a region
filter = df["Region"]=="srce"

# filtered data description 
df.where(filter, inplace=False).describe() 


,Lead (Pb) mg/L,Copper (Cu) mg/L
count,312.000000,312.000000
mean,0.006145,0.277786
std,0.036554,0.542652
min,0.000000,0.001800
25%,0.000000,0.066000
50%,0.000700,0.130000
75%,0.002100,0.250000
max,0.450000,4.300000


In [16]:
# for more than one region
# making boolean series for a region
filter = df["Region"].isin(["srce","csap_en"])

# filtered data description 
df.where(filter, inplace=False).describe() 

,Lead (Pb) mg/L,Copper (Cu) mg/L
count,782.000000,782.000000
mean,0.009734,0.558019
std,0.039038,1.389291
min,0.000000,0.000000
25%,0.000100,0.082525
50%,0.001200,0.160800
75%,0.005175,0.394650
max,0.597300,15.280000


In [17]:
# maximum Lead
df[df['Lead (Pb) mg/L'] == df['Lead (Pb) mg/L'].max()]

,School Name,Location,Lead (Pb) mg/L,Lead Result,Copper (Cu) mg/L,Copper Result,Region
Sample ID,,,,,,,
C0G5324/NAG927,Central Kings Rural High School,C120 - Gym Office - 100022422,20.0,Exceeds Limit,42.0,Exceeds Limit,avrce


In [18]:
# maximum copper
df[df['Copper (Cu) mg/L'] == df['Copper (Cu) mg/L'].max()]

,School Name,Location,Lead (Pb) mg/L,Lead Result,Copper (Cu) mg/L,Copper Result,Region
Sample ID,,,,,,,
MYR801,John Martin Junior High School,Boys WR tap 1 by learning centre,2.2,Exceeds Limit,53.0,Exceeds Limit,hrce
